# Exploración y Limpieza de Datos de Defunciones

1. **Carga de Datos y Tipificación**  
   - Lectura del archivo `defunciones.csv` con tipos de dato apropiados.  
   - Renombre de columnas relevantes y conversión a categorías.

2. **Resumen Descriptivo**  
   - Se muestran dimensiones del dataset, conteo de valores nulos y estadísticas numéricas.

3. **Mapeo ICD-10 (CIE.csv)**  
   - Lectura de códigos para simplificar la columna de causas de defunción.

4. **Proceso de Limpieza Final**  
   - Eliminación de filas con valores faltantes.  
   - Corrección de categorías desconocidas en `ocupacion`.  
   - Creación de la columna `age_group`.  
   - Exportación final a `defunciones_simplified.csv`.

In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Carga y tipificación inicial
df = pd.read_csv("defunciones.csv")
col_types = {col: str for col in df.columns}
col_types["Añoreg"] = int
col_types["Edadif"] = int

# Selección y renombre de columnas
defunciones = pd.read_csv("defunciones.csv", dtype=col_types)
defunciones = defunciones[
    ["Depocu","Mupocu","Sexo","Añoreg","Edadif","Perdif","Puedif","Escodif","Ciuodif","Caudef","Asist","Ocur"]
]
defunciones.rename(columns={
    "Depocu": "departamento",
    "Mupocu": "municipio",
    "Sexo": "sexo",
    "Añoreg": "año_registrado",
    "Edadif": "edad",
    "Perdif": "periodo",
    "Puedif": "etnia",
    "Escodif": "escolaridad",
    "Ciuodif": "ocupacion",
    "Caudef": "causa",
    "Asist": "asistencia",
    "Ocur": "lugar"
}, inplace=True)
defunciones.sort_values(by="año_registrado", inplace=True)
defunciones.to_csv("defunciones_clean.csv", index=False)

# Conversión de columnas a categoría
for c in defunciones.columns:
    if c not in ["año_registrado", "edad"]:
        defunciones[c] = defunciones[c].astype(str).str.strip().astype("category")

# Recarga del CSV limpio con tipos
column_types = {
    "departamento": "category",
    "municipio": "category",
    "sexo": "category",
    "año_registrado": "category",
    "edad": "int64",
    "periodo": "category",
    "etnia": "category",
    "escolaridad": "category",
    "ocupacion": "category",
    "causa": "category",
    "asistencia": "category",
    "lugar": "category",
}
defunciones = pd.read_csv("defunciones_clean.csv", dtype=column_types)
display(defunciones.head())

# Estadísticas básicas
print("Dimensiones (filas, columnas):", defunciones.shape)
print("\nNulos por columna:")
display(defunciones.isna().sum())
print("\nResumen estadístico (numérico):")
display(defunciones.describe())

# Lectura y mapeo de códigos ICD-10
def load_codes_csv(file_path):
    df_cie = pd.read_csv(file_path)
    codes_dict = {}
    for _, row in df_cie.iterrows():
        codigo = str(row["codigo"]).strip()
        descripcion = str(row["descripcion"]).strip()
        codes_dict[codigo] = descripcion
    return codes_dict

def simplify_icd10(code, code_map):
    if pd.isna(code):
        return "Unknown"
    code_clean = code.strip()
    if code_clean in code_map:
        return code_map[code_clean]
    prefix = code_clean.split(".")[0]
    for c in code_map:
        if c.startswith(prefix):
            return code_map[c]
    return "Other"

def split(defunciones_df, code_map):
    data = defunciones_df.dropna().copy()
    data["causa_simplificada"] = data["causa"].apply(lambda x: simplify_icd10(x, code_map))
    data = data[data["causa_simplificada"] != "Other"]
    data.drop(columns=["causa"], inplace=True)
    data.rename(columns={"causa_simplificada": "causa"}, inplace=True)
    mode = data["ocupacion"].mode()[0]
    data["ocupacion"] = data["ocupacion"].replace(["NEOG", "IGNORADO"], mode)
    data["age_group"] = pd.cut(
        data["edad"],
        bins=[0, 18, 35, 50, 65, float("inf")],
        labels=["0-18","19-35","36-50","51-65","65+"],
    )
    data.drop(columns=["edad"], inplace=True)
    data.to_csv("defunciones_simplified.csv", index=False)
    return data

# Aplicación final
code_map = load_codes_csv("CIE.csv")
causes = split(defunciones, code_map)
display(causes.head())

/var/folders/jq/zj1d7dc15dd57tj80xs_lvxr0000gn/T/ipykernel_68314/4293297565.py:7: DtypeWarning: Columns (18,28,29) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("defunciones.csv")


,departamento,municipio,sexo,año_registrado,edad,periodo,etnia,escolaridad,ocupacion,causa,asistencia,lugar
0,17.0,1703,1.0,2012,28,3.0,NaN,1.0,NaN,M329,5.0,6.0
1,1.0,0101,2.0,2012,88,3.0,NaN,2.0,NaN,E142,1.0,6.0
2,1.0,0101,2.0,2012,74,3.0,NaN,2.0,NaN,E039,1.0,1.0
3,1.0,0101,2.0,2012,43,3.0,NaN,2.0,NaN,E149,1.0,6.0
4,1.0,0101,2.0,2012,88,3.0,NaN,2.0,NaN,E119,1.0,6.0


Dimensiones (filas, columnas): (855407, 12)

Nulos por columna:


departamento          0
municipio             0
sexo                  0
año_registrado        0
edad                  0
periodo               0
etnia             72657
escolaridad           0
ocupacion         72657
causa                 0
asistencia            0
lugar                 0
dtype: int64


Resumen estadístico (numérico):


,edad
count,855407.000000
mean,61.549722
std,82.407296
min,0.000000
25%,34.000000
50%,62.000000
75%,78.000000
max,999.000000


/var/folders/jq/zj1d7dc15dd57tj80xs_lvxr0000gn/T/ipykernel_68314/4293297565.py:93: FutureWarning: The behavior of Series.replace (and DataFrame.replace) with CategoricalDtype is deprecated. In a future version, replace will only be used for cases that preserve the categories. To change the categories, use ser.cat.rename_categories instead.
  data["ocupacion"] = data["ocupacion"].replace(["NEOG", "IGNORADO"], mode)


,departamento,municipio,sexo,año_registrado,periodo,etnia,escolaridad,ocupacion,asistencia,lugar,causa,age_group
70905,14.0,1415,1.0,2013,3.0,1.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70906,14.0,1411,1.0,2013,3.0,1.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70907,16.0,1601,1.0,2013,3.0,1.0,9.0,92,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70908,1.0,0116,1.0,2013,3.0,9.0,1.0,61,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
70909,12.0,1219,1.0,2013,3.0,4.0,1.0,92,5.0,6.0,"Úlcera gástrica, no especificada como aguda ni...",65+
